# Funciones útiles

In [39]:
%matplotlib notebook

In [40]:
from distutils.spawn import find_executable

from matplotlib.font_manager import *
from matplotlib.collections import *
from matplotlib.patches import *
from matplotlib.pylab import *
from matplotlib import colors

import seaborn

rem = 16

seaborn.set(context='notebook', style='darkgrid')

ioff()

rc('lines', linewidth=1)
rc('font', family='serif')
rc('font', size=rem)
rc('axes', titlepad=1.500*rem)
rc('axes', titlesize=1.728*rem)
rc('axes', labelsize=1.200*rem)
rc('legend', fontsize=1.000*rem)
rc('xtick', labelsize=0.833*rem)
rc('ytick', labelsize=0.833*rem)

if find_executable('latex'):
    rc('text', usetex=True)

In [41]:
import numpy
from numpy import *
from scipy.integrate import simps

## Segunda derivada

In [42]:
def second_derivative(data, h):
    out = numpy.array(data)
    for i in range(1, len(data) - 1):
        out[i] = data[i + 1] - 2 * data[i] + data[i - 1]
    out = out / (h * h)
    out[0] = out[1]
    out[-1] = out[-2]

    out[0] = (data[2] - 2 * data[1] + data[0])/(h**2)
    out[-1] = (data[-1] - 2 * data[-1-1] + data[-1-2])/(h**2)
    
    return out

## $\theta$ de Heaviside

In [43]:
def heaviside(x):
    out = numpy.zeros_like(x)
    out[x >= 0] = 1.0
    
    return out

## Obtención de las matrices $T$, $V$ y $S$

In [44]:
def get_Matrices(values_n, data_basis, data_potential, x):
    S = numpy.zeros(shape=(len(values_n), len(values_n)))
    T = numpy.zeros(shape=(len(values_n), len(values_n)))
    V = numpy.zeros(shape=(len(values_n), len(values_n)))
    
    for m, mval in enumerate(values_n):
        for n, nval in enumerate(values_n):
            S[m, n] = float("%0.4f" % simps(numpy.conjugate(
                data_basis[mval]) * data_basis[nval], x).real)
    
            T[m, n] = simps(numpy.conjugate(
                data_basis[mval]) * (-0.5) * second_derivative(
                data_basis[nval], x[1] - x[0]), x).real
        
            V[m, n] = simps(numpy.conjugate(
                data_basis[mval]) * data_potential * data_basis[nval], x).real
            
    return S, T, V

## Obtención de los autovalores y autovectores

In [45]:
def eigen_vec_vals(S, values_n, ERROR, H):
    if numpy.allclose(S, numpy.identity(len(values_n)), atol=ERROR):
        E, C = numpy.linalg.eig(H)
        C_prime = C.copy()
    else:
        diag, U = numpy.linalg.eig(S)
        Sd = numpy.diag(diag)
        V = numpy.dot(U, numpy.linalg.inv(Sd)**0.5)

        H_prime = numpy.dot(numpy.dot(numpy.conjugate(V.T), H), V)
        E, C_prime = numpy.linalg.eig(H_prime)
        C = numpy.dot(V, C_prime)
        
    return E, C, C_prime

# Solución

In [8]:
def potential(x, Vo=1.0, width=1.0):
    return Vo * heaviside(x - width / 2 + 0.25*width) * (1 - heaviside(x - width / 2 - 0.25*width))

def basis_n_K(n, K, width=1.0, x=[1.0]):
    return sqrt(1 / width) * exp(1.0j * (K + 2 * pi * n / width) * x)

In [9]:
width = 1.0
ndiv = 100
ndiv_k = 50
N = 15

x = numpy.linspace(0, width, ndiv)

data_potential = potential(x, Vo=50, width=width)

values_n = [0]
for n in range(1, N+1):
    values_n.append(n)
    values_n.append(-n)

K_arr = numpy.linspace(-pi/width, pi/width, ndiv_k)

A = {}
for m in values_n:
    A[m] = simps(
        numpy.conjugate(basis_n_K(m, 0, width=width, x=x)) * basis_n_K(m, 0, width=width, x=x),
        x
    ).real
    
T = numpy.zeros(shape=(len(values_n), len(values_n), len(K_arr)))
V = numpy.zeros(shape=(len(values_n), len(values_n), len(K_arr)))
S = numpy.zeros(shape=(len(values_n), len(values_n), len(K_arr)))

for k, K in enumerate(K_arr):
    data_basis = {}
    
    for m in values_n:
        data_basis[m] = basis_n_K(m, K, width=width, x=x) / sqrt(A[m])

    S_, T_, V_ = get_Matrices(values_n, data_basis, data_potential, x)
    S[:, :, k] = S_
    T[:, :, k] = T_
    V[:, :, k] = V_
H = T + V

energies = list()
uenergies = list()
amplitudes = list()
for k, K in enumerate(K_arr):
    eigenenergies, eigenvectors = numpy.linalg.eig(H[:, :, k])
    uenergies.append(eigenenergies)
    amplitudes.append(eigenvectors)
    energies.append(sorted(eigenenergies))
    
energies = numpy.array(energies)

In [36]:
uenergies = numpy.array(uenergies)
amplitudes = numpy.array(amplitudes)

for m in enumerate(values_n):
    for n, eigenenergy in enumerate(uenergies[:, m]):
        eigenvector = amplitudes[n, :, m]
        
        for K in K_arr:
            y = numpy.zeros(shape=x.shape)
            
            for c in eigenvector:
                print("***")
                print("E = ", eigenenergy)
                print("K = ", K)
                print("E = ", eigenenergy)
                y = y + c * basis_n_K(m, K, width=width, x=x)
                
                if numpy.random.rand() > 0.99:
                    plot(x, numpy.abs(y * y.conj()))
                    show()
                    time.sleep(5)

TypeError: can't multiply sequence by non-int of type 'float'

In [31]:
K_arr.shape

(50,)

In [32]:
uenergies.shape

(50, 31)

In [33]:
amplitudes.shape

(50, 31, 31)

In [34]:
eigenvector.shape

(31,)

In [38]:
fibBands_PAGE2 = figure(1)
ax = fibBands_PAGE2.add_subplot(111)

for n in range(6):
    ax.plot(K_arr * width / pi, energies[:, n], lw=2)

    if n==0:
        min1 = max(energies[:, n])
    else:
        max1 = min(energies[:, n])
        ax.add_patch(Rectangle((min(K_arr) * width / numpy.pi, min1),
                                           2 * max(K_arr) * width / numpy.pi, max1-min1,
                                           facecolor="green",
                                           alpha=0.2, lw=0))
    min1 = max(energies[:, n])

ax.set_xlabel(r"$K \cdot width/\pi$")
ax.set_ylabel(r"$E$")
ax_potential = ax.twinx()
lim_min = min([ax.get_ylim()[0], ax_potential.get_ylim()[0]])
lim_max = max([ax.get_ylim()[1], ax_potential.get_ylim()[1]])
ax_potential.set_ylim(lim_min, lim_max)
ax.set_ylim(lim_min, lim_max)

newx = numpy.sort(numpy.concatenate((x, -x)))
hatch = "/"
alpha=0.3
fcol = "red"
ax_potential.fill_between(x, 0, data_potential, hatch=hatch, facecolor=fcol, alpha=alpha)
ax_potential.fill_between(-x, 0, data_potential, hatch=hatch, facecolor=fcol, alpha=alpha)

ax_potential.set_ylabel(r"$V\left(x\right)$",
                        color="r")
fibBands_PAGE2.tight_layout()
ax.grid(False)
ax_potential.grid(False)
show()

/opt/conda/envs/kroning-penney-model/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [146]:
uenergies = numpy.array(uenergies)
uenergies.shape

(50, 50, 31)